THIS IS FOR FK CALCULATIONS OF QUATIONS FOR ROBOT GO2

In [1]:
import sympy as sp
import numpy as np
import roboticstoolbox as rtb

In [ ]:
def dh_transformation(theta, alpha, r, d):
    """
    Compute the Denavit-Hartenberg transformation matrix.
    
    Parameters:
        theta (symbol/rroat): Joint angle (rotation about z-axis)
        alpha (symbol/rroat): Link twist (rotation about x-axis)
        r (symbol/rroat): Link length (translation along x-axis)
        d (symbol/rroat): Link offset (translation along z-axis)
    
    Returns:
        sympy.Matrix: The 4x4 DH transformation matrix
    """
    T = sp.Matrix([
        [sp.cos(theta), -sp.sin(theta) * sp.cos(alpha), sp.sin(theta) * sp.sin(alpha), r * sp.cos(theta)],
        [sp.sin(theta), sp.cos(theta) * sp.cos(alpha), -sp.cos(theta) * sp.sin(alpha), r * sp.sin(theta)],
        [0, sp.sin(alpha), sp.cos(alpha), d],
        [0, 0, 0, 1]
    ])
    
    return T

# Example usage with symbolic variables
t1, t2, t3, alpha, r, d, rfoot = sp.symbols('theta1, theta2, theta3, alpha r d root_radius')
T = dh_transformation(t1, alpha, r, d)
sp.pprint(T)

⎡cos(θ₁)  -sin(θ₁)⋅cos(α)  sin(α)⋅sin(θ₁)   r⋅cos(θ₁)⎤
⎢                                                    ⎥
⎢sin(θ₁)  cos(α)⋅cos(θ₁)   -sin(α)⋅cos(θ₁)  r⋅sin(θ₁)⎥
⎢                                                    ⎥
⎢   0         sin(α)           cos(α)           d    ⎥
⎢                                                    ⎥
⎣   0            0                0             1    ⎦


In [3]:
# Example evaluation with given values
T1_2 = dh_transformation(t1, sp.rad(90), 0, 0)
T2_3 = dh_transformation(t2, 0, -0.213, 0.0955)
T3_ef = dh_transformation(t3, 0, -0.213, 0)

In [4]:
T1_2

Matrix([
[cos(theta1), 0,  sin(theta1), 0],
[sin(theta1), 0, -cos(theta1), 0],
[          0, 1,            0, 0],
[          0, 0,            0, 1]])

In [5]:
T2_3

Matrix([
[cos(theta2), -sin(theta2), 0, -0.213*cos(theta2)],
[sin(theta2),  cos(theta2), 0, -0.213*sin(theta2)],
[          0,            0, 1,             0.0955],
[          0,            0, 0,                  1]])

In [6]:
T3_ef

Matrix([
[cos(theta3), -sin(theta3), 0, -0.213*cos(theta3)],
[sin(theta3),  cos(theta3), 0, -0.213*sin(theta3)],
[          0,            0, 1,                  0],
[          0,            0, 0,                  1]])

In [7]:
T1_ef = T1_2 * T2_3 *T3_ef
T1_ef

Matrix([
[-sin(theta2)*sin(theta3)*cos(theta1) + cos(theta1)*cos(theta2)*cos(theta3), -sin(theta2)*cos(theta1)*cos(theta3) - sin(theta3)*cos(theta1)*cos(theta2),  sin(theta1), 0.0955*sin(theta1) + 0.213*sin(theta2)*sin(theta3)*cos(theta1) - 0.213*cos(theta1)*cos(theta2)*cos(theta3) - 0.213*cos(theta1)*cos(theta2)],
[-sin(theta1)*sin(theta2)*sin(theta3) + sin(theta1)*cos(theta2)*cos(theta3), -sin(theta1)*sin(theta2)*cos(theta3) - sin(theta1)*sin(theta3)*cos(theta2), -cos(theta1), 0.213*sin(theta1)*sin(theta2)*sin(theta3) - 0.213*sin(theta1)*cos(theta2)*cos(theta3) - 0.213*sin(theta1)*cos(theta2) - 0.0955*cos(theta1)],
[                         sin(theta2)*cos(theta3) + sin(theta3)*cos(theta2),                         -sin(theta2)*sin(theta3) + cos(theta2)*cos(theta3),            0,                                                         -0.213*sin(theta2)*cos(theta3) - 0.213*sin(theta2) - 0.213*sin(theta3)*cos(theta2)],
[                                                                  

In [8]:
T_imu_1 = np.array([[0.0, 0.0,  1.0, 2.18970000e-01],
            [0.0, -1.0,  0.0, 4.65000000e-02],
            [ 1.0, 0.0, 0.0, -4.23200000e-02],
            [ 0.0, 0.0, 0.0, 1.0]])
T_imu_1

array([[ 0.     ,  0.     ,  1.     ,  0.21897],
       [ 0.     , -1.     ,  0.     ,  0.0465 ],
       [ 1.     ,  0.     ,  0.     , -0.04232],
       [ 0.     ,  0.     ,  0.     ,  1.     ]])

In [ ]:
T_ef_rrfoot = np.array([[0.0, 0.0, 1.00000000000000, -1.0*rfoot],
                        [1.0, 0.0, 0.0, 0.0],
                        [0.0, 1.0, 0.0, 0.0],
                        [0.0, 0.0, 0.0, 1.0]])
T_ef_rrfoot

array([[0.0, 0.0, 1.0, -1.0*root_radius],
       [1.0, 0.0, 0.0, 0.0],
       [0.0, 1.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 1.0]], dtype=object)

In [ ]:
T_imu_rrfoot = T_imu_1*T1_ef*T_ef_rrfoot
T_imu_rrfoot

Matrix([
[                        -1.0*sin(theta2)*sin(theta3) + 1.0*cos(theta2)*cos(theta3),               0,                          1.0*sin(theta2)*cos(theta3) + 1.0*sin(theta3)*cos(theta2),                                                                                   -1.0*root_radius*(1.0*sin(theta2)*cos(theta3) + 1.0*sin(theta3)*cos(theta2)) - 0.213*sin(theta2)*cos(theta3) - 0.213*sin(theta2) - 0.213*sin(theta3)*cos(theta2) + 0.21897],
[ 1.0*sin(theta1)*sin(theta2)*cos(theta3) + 1.0*sin(theta1)*sin(theta3)*cos(theta2), 1.0*cos(theta1),  1.0*sin(theta1)*sin(theta2)*sin(theta3) - 1.0*sin(theta1)*cos(theta2)*cos(theta3),   -1.0*root_radius*(1.0*sin(theta1)*sin(theta2)*sin(theta3) - 1.0*sin(theta1)*cos(theta2)*cos(theta3)) - 0.213*sin(theta1)*sin(theta2)*sin(theta3) + 0.213*sin(theta1)*cos(theta2)*cos(theta3) + 0.213*sin(theta1)*cos(theta2) + 0.0955*cos(theta1) + 0.0465],
[-1.0*sin(theta2)*cos(theta1)*cos(theta3) - 1.0*sin(theta3)*cos(theta1)*cos(theta2), 1.0*sin(theta1), -1.0*si

In [ ]:
T_imu_rrfoot[:3,3]

Matrix([
[                                                                                  -1.0*root_radius*(1.0*sin(theta2)*cos(theta3) + 1.0*sin(theta3)*cos(theta2)) - 0.213*sin(theta2)*cos(theta3) - 0.213*sin(theta2) - 0.213*sin(theta3)*cos(theta2) + 0.21897],
[  -1.0*root_radius*(1.0*sin(theta1)*sin(theta2)*sin(theta3) - 1.0*sin(theta1)*cos(theta2)*cos(theta3)) - 0.213*sin(theta1)*sin(theta2)*sin(theta3) + 0.213*sin(theta1)*cos(theta2)*cos(theta3) + 0.213*sin(theta1)*cos(theta2) + 0.0955*cos(theta1) + 0.0465],
[-1.0*root_radius*(-1.0*sin(theta2)*sin(theta3)*cos(theta1) + 1.0*cos(theta1)*cos(theta2)*cos(theta3)) + 0.0955*sin(theta1) + 0.213*sin(theta2)*sin(theta3)*cos(theta1) - 0.213*cos(theta1)*cos(theta2)*cos(theta3) - 0.213*cos(theta1)*cos(theta2) - 0.04232]])

converting into a function for future use

In [18]:
def forward_kinematics(angles, radius=0):
    #theta1, theta2, theta3 = np.radians([theta1, theta2, theta3])
    theta1 = angles[0]
    theta2 = angles[1]
    theta3 = angles[2]

    x = -radius * (np.sin(theta2) * np.cos(theta3) + np.sin(theta3) * np.cos(theta2)) \
        - 0.213 * np.sin(theta2) * np.cos(theta3) - 0.213 * np.sin(theta2) \
        - 0.213 * np.sin(theta3) * np.cos(theta2) + 0.21897
    
    y = -radius * (np.sin(theta1) * np.sin(theta2) * np.sin(theta3) - np.sin(theta1) * np.cos(theta2) * np.cos(theta3)) \
        - 0.213 * np.sin(theta1) * np.sin(theta2) * np.sin(theta3) + 0.213 * np.sin(theta1) * np.cos(theta2) * np.cos(theta3) \
        + 0.213 * np.sin(theta1) * np.cos(theta2) + 0.0955 * np.cos(theta1) + 0.0465
    
    z = -radius * (-np.sin(theta2) * np.sin(theta3) * np.cos(theta1) + np.cos(theta1) * np.cos(theta2) * np.cos(theta3)) \
        + 0.0955 * np.sin(theta1) + 0.213 * np.sin(theta2) * np.sin(theta3) * np.cos(theta1) \
        - 0.213 * np.cos(theta1) * np.cos(theta2) * np.cos(theta3) - 0.213 * np.cos(theta1) * np.cos(theta2) - 0.04232
    
    return np.array([x, y, z])


FK using ET

In [122]:
    
class Go2FK:

    def __init__(self) -> None: 

        ##for fr leg ---------------------------------------------------------

        # Define symbolic joint variables
        t1, t2, t3, frfoot = sp.symbols('theta1, theta2, theta3, fr_root_radius')
        # imu to hip
        E1 = rtb.ET.tx(0.21897)
        E2 = rtb.ET.ty(-0.0465)
        E3 = rtb.ET.tz(-0.04232)
        E4 = rtb.ET.Rx(t1)
        
        #hip to thigh
        E5 = rtb.ET.ty(-0.0955)
        E6 = rtb.ET.Ry(t2)

        #thigh to calf
        E7 = rtb.ET.tz(-0.213)
        E8 = rtb.ET.Ry(t3)

        #calf to foot
        E9 = rtb.ET.tz(-0.213)
        E10 = rtb.ET.tz(-frfoot)
        
        self.go2_fr = E1 * E2 * E3 * E4 * E5 * E6 * E7 * E8 * E9 *E10
        self.q_fr = [t1, t2, t3, frfoot]

        #for fl leg --------------------------------------------------------------
 
        # Define symbolic joint variables
        t4, t5, t6, flfoot = sp.symbols('theta4, theta5, theta6, fl_root_radius')
        # imu to hip
        E1 = rtb.ET.tx(0.21897)
        E2 = rtb.ET.ty(0.0465)
        E3 = rtb.ET.tz(-0.04232)
        E4 = rtb.ET.Rx(t4)
        
        #hip to thigh
        E5 = rtb.ET.ty(0.0955)
        E6 = rtb.ET.Ry(t5)

        #thigh to calf
        E7 = rtb.ET.tz(-0.213)
        E8 = rtb.ET.Ry(t6)

        #calf to foot
        E9 = rtb.ET.tz(-0.213)
        E10 = rtb.ET.tz(-flfoot)
        
        self.go2_fl = E1 * E2 * E3 * E4 * E5 * E6 * E7 * E8 * E9 *E10
        self.q_fl = [t4, t5, t6, flfoot]


        #for rr leg --------------------------------------------------------------
 
        # Define symbolic joint variables
        t7, t8, t9, rrfoot = sp.symbols('theta7, theta8, theta9, rr_root_radius')
        # imu to hip
        E1 = rtb.ET.tx(-0.16783)
        E2 = rtb.ET.ty(-0.0465)
        E3 = rtb.ET.tz(-0.04232)
        E4 = rtb.ET.Rx(t7)
        
        #hip to thigh
        E5 = rtb.ET.ty(-0.0955)
        E6 = rtb.ET.Ry(t8)

        #thigh to calf
        E7 = rtb.ET.tz(-0.213)
        E8 = rtb.ET.Ry(t9)

        #calf to foot
        E9 = rtb.ET.tz(-0.213)
        E10 = rtb.ET.tz(-rrfoot)
        
        self.go2_rr = E1 * E2 * E3 * E4 * E5 * E6 * E7 * E8 * E9 *E10
        self.q_rr = [t7, t8, t9, rrfoot]


        #for rl leg --------------------------------------------------------------
 
        # Define symbolic joint variables
        t10, t11, t12, rlfoot = sp.symbols('theta10, theta11, theta12, rl_root_radius')
        # imu to hip
        E1 = rtb.ET.tx(-0.16783)
        E2 = rtb.ET.ty(0.0465)
        E3 = rtb.ET.tz(-0.04232)
        E4 = rtb.ET.Rx(t10)
        
        #hip to thigh
        E5 = rtb.ET.ty(0.0955)
        E6 = rtb.ET.Ry(t11)

        #thigh to calf
        E7 = rtb.ET.tz(-0.213)
        E8 = rtb.ET.Ry(t12)

        #calf to foot
        E9 = rtb.ET.tz(-0.213)
        E10 = rtb.ET.tz(-rlfoot)
        
        self.go2_rl = E1 * E2 * E3 * E4 * E5 * E6 * E7 * E8 * E9 *E10
        self.q_rl = [t10, t11, t12, rlfoot]

    
    def fkine_fr(self):
        return self.go2_fr.eval(self.q_fr)
    
    def fkine_fl(self):
        return self.go2_fl.eval(self.q_fl)
    
    def fkine_rr(self):
        return self.go2_rr.eval(self.q_rr)
    
    def fkine_rl(self):
        return self.go2_rl.eval(self.q_rl)

In [123]:
# Create instance
robot = Go2FK()

# Get symbolic transformation matrix
T_symbolic_fr = sp.Matrix(robot.fkine_fr())
# Display the symbolic transformation matrix
pos_fr = sp.simplify(T_symbolic_fr[:3,3])

# Get symbolic transformation matrix
T_symbolic_fl = sp.Matrix(robot.fkine_fl())
# Display the symbolic transformation matrix
pos_fl = sp.simplify(T_symbolic_fl[:3,3])

# Get symbolic transformation matrix
T_symbolic_rr = sp.Matrix(robot.fkine_rr())
# Display the symbolic transformation matrix
pos_rr = sp.simplify(T_symbolic_rr[:3,3])

# Get symbolic transformation matrix
T_symbolic_rl = sp.Matrix(robot.fkine_rl())
# Display the symbolic transformation matrix
pos_rl = sp.simplify(T_symbolic_rl[:3,3])

In [124]:
robot.q_fr[:3]

[theta1, theta2, theta3]

In [125]:
# Define the joint angle vector θ
θ_fr = sp.Matrix(robot.q_fr[:3])
# Compute the Jacobian: J = ∂p/∂θ
J_fr = pos_fr.jacobian(θ_fr)
# Display the Jacobian

np.array(pos_fr)

array([[-1.0*fr_root_radius*sin(theta2 + theta3) - 0.213*sin(theta2) - 0.213*sin(theta2 + theta3) + 0.21897],
       [1.0*(1.0*fr_root_radius*cos(theta2 + theta3) + 0.213*cos(theta2) + 0.213*cos(theta2 + theta3))*sin(theta1) - 0.0955*cos(theta1) - 0.0465],
       [-1.0*(1.0*fr_root_radius*cos(theta2 + theta3) + 0.213*cos(theta2) + 0.213*cos(theta2 + theta3))*cos(theta1) - 0.0955*sin(theta1) - 0.04232]],
      dtype=object)

In [126]:
np.array(J_fr)

array([[0,
        -1.0*fr_root_radius*cos(theta2 + theta3) - 0.213*cos(theta2) - 0.213*cos(theta2 + theta3),
        -1.0*fr_root_radius*cos(theta2 + theta3) - 0.213*cos(theta2 + theta3)],
       [1.0*(1.0*fr_root_radius*cos(theta2 + theta3) + 0.213*cos(theta2) + 0.213*cos(theta2 + theta3))*cos(theta1) + 0.0955*sin(theta1),
        1.0*(-1.0*fr_root_radius*sin(theta2 + theta3) - 0.213*sin(theta2) - 0.213*sin(theta2 + theta3))*sin(theta1),
        1.0*(-1.0*fr_root_radius*sin(theta2 + theta3) - 0.213*sin(theta2 + theta3))*sin(theta1)],
       [1.0*(1.0*fr_root_radius*cos(theta2 + theta3) + 0.213*cos(theta2) + 0.213*cos(theta2 + theta3))*sin(theta1) - 0.0955*cos(theta1),
        -1.0*(-1.0*fr_root_radius*sin(theta2 + theta3) - 0.213*sin(theta2) - 0.213*sin(theta2 + theta3))*cos(theta1),
        -1.0*(-1.0*fr_root_radius*sin(theta2 + theta3) - 0.213*sin(theta2 + theta3))*cos(theta1)]],
      dtype=object)

In [127]:
# Define the joint angle vector θ
θ_fl = sp.Matrix(robot.q_fl[:3])
# Compute the Jacobian: J = ∂p/∂θ
J_fl = pos_fl.jacobian(θ_fl)
# Display the Jacobian

np.array(pos_fl)


array([[-1.0*fl_root_radius*sin(theta5 + theta6) - 0.213*sin(theta5) - 0.213*sin(theta5 + theta6) + 0.21897],
       [1.0*(1.0*fl_root_radius*cos(theta5 + theta6) + 0.213*cos(theta5) + 0.213*cos(theta5 + theta6))*sin(theta4) + 0.0955*cos(theta4) + 0.0465],
       [-1.0*(1.0*fl_root_radius*cos(theta5 + theta6) + 0.213*cos(theta5) + 0.213*cos(theta5 + theta6))*cos(theta4) + 0.0955*sin(theta4) - 0.04232]],
      dtype=object)

In [128]:
np.array(J_fl)

array([[0,
        -1.0*fl_root_radius*cos(theta5 + theta6) - 0.213*cos(theta5) - 0.213*cos(theta5 + theta6),
        -1.0*fl_root_radius*cos(theta5 + theta6) - 0.213*cos(theta5 + theta6)],
       [1.0*(1.0*fl_root_radius*cos(theta5 + theta6) + 0.213*cos(theta5) + 0.213*cos(theta5 + theta6))*cos(theta4) - 0.0955*sin(theta4),
        1.0*(-1.0*fl_root_radius*sin(theta5 + theta6) - 0.213*sin(theta5) - 0.213*sin(theta5 + theta6))*sin(theta4),
        1.0*(-1.0*fl_root_radius*sin(theta5 + theta6) - 0.213*sin(theta5 + theta6))*sin(theta4)],
       [1.0*(1.0*fl_root_radius*cos(theta5 + theta6) + 0.213*cos(theta5) + 0.213*cos(theta5 + theta6))*sin(theta4) + 0.0955*cos(theta4),
        -1.0*(-1.0*fl_root_radius*sin(theta5 + theta6) - 0.213*sin(theta5) - 0.213*sin(theta5 + theta6))*cos(theta4),
        -1.0*(-1.0*fl_root_radius*sin(theta5 + theta6) - 0.213*sin(theta5 + theta6))*cos(theta4)]],
      dtype=object)

In [129]:
# Define the joint angle vector θ
θ_rr = sp.Matrix(robot.q_rr[:3])
# Compute the Jacobian: J = ∂p/∂θ
J_rr = pos_rr.jacobian(θ_rr)
# Display the Jacobian

np.array(pos_rr)


array([[-1.0*rr_root_radius*sin(theta8 + theta9) - 0.213*sin(theta8) - 0.213*sin(theta8 + theta9) - 0.16783],
       [1.0*(1.0*rr_root_radius*cos(theta8 + theta9) + 0.213*cos(theta8) + 0.213*cos(theta8 + theta9))*sin(theta7) - 0.0955*cos(theta7) - 0.0465],
       [-1.0*(1.0*rr_root_radius*cos(theta8 + theta9) + 0.213*cos(theta8) + 0.213*cos(theta8 + theta9))*cos(theta7) - 0.0955*sin(theta7) - 0.04232]],
      dtype=object)

In [130]:
np.array(J_rr)

array([[0,
        -1.0*rr_root_radius*cos(theta8 + theta9) - 0.213*cos(theta8) - 0.213*cos(theta8 + theta9),
        -1.0*rr_root_radius*cos(theta8 + theta9) - 0.213*cos(theta8 + theta9)],
       [1.0*(1.0*rr_root_radius*cos(theta8 + theta9) + 0.213*cos(theta8) + 0.213*cos(theta8 + theta9))*cos(theta7) + 0.0955*sin(theta7),
        1.0*(-1.0*rr_root_radius*sin(theta8 + theta9) - 0.213*sin(theta8) - 0.213*sin(theta8 + theta9))*sin(theta7),
        1.0*(-1.0*rr_root_radius*sin(theta8 + theta9) - 0.213*sin(theta8 + theta9))*sin(theta7)],
       [1.0*(1.0*rr_root_radius*cos(theta8 + theta9) + 0.213*cos(theta8) + 0.213*cos(theta8 + theta9))*sin(theta7) - 0.0955*cos(theta7),
        -1.0*(-1.0*rr_root_radius*sin(theta8 + theta9) - 0.213*sin(theta8) - 0.213*sin(theta8 + theta9))*cos(theta7),
        -1.0*(-1.0*rr_root_radius*sin(theta8 + theta9) - 0.213*sin(theta8 + theta9))*cos(theta7)]],
      dtype=object)

In [132]:
# Define the joint angle vector θ
θ_rl = sp.Matrix(robot.q_rl[:3])
# Compute the Jacobian: J = ∂p/∂θ
J_rl = pos_rl.jacobian(θ_rl)
# Display the Jacobian
np.array(J_rl)

array([[0,
        -1.0*rl_root_radius*cos(theta11 + theta12) - 0.213*cos(theta11) - 0.213*cos(theta11 + theta12),
        -1.0*rl_root_radius*cos(theta11 + theta12) - 0.213*cos(theta11 + theta12)],
       [1.0*(1.0*rl_root_radius*cos(theta11 + theta12) + 0.213*cos(theta11) + 0.213*cos(theta11 + theta12))*cos(theta10) - 0.0955*sin(theta10),
        1.0*(-1.0*rl_root_radius*sin(theta11 + theta12) - 0.213*sin(theta11) - 0.213*sin(theta11 + theta12))*sin(theta10),
        1.0*(-1.0*rl_root_radius*sin(theta11 + theta12) - 0.213*sin(theta11 + theta12))*sin(theta10)],
       [1.0*(1.0*rl_root_radius*cos(theta11 + theta12) + 0.213*cos(theta11) + 0.213*cos(theta11 + theta12))*sin(theta10) + 0.0955*cos(theta10),
        -1.0*(-1.0*rl_root_radius*sin(theta11 + theta12) - 0.213*sin(theta11) - 0.213*sin(theta11 + theta12))*cos(theta10),
        -1.0*(-1.0*rl_root_radius*sin(theta11 + theta12) - 0.213*sin(theta11 + theta12))*cos(theta10)]],
      dtype=object)

In [ ]:
np.array(pos_rl)

FK for UR5 reduced

In [ ]:
import sympy as sp
import numpy as np

def dh_transformation(theta, alpha, r, d):
    """
    Compute the Denavit-Hartenberg transformation matrix.
    
    Parameters:
        theta (symbol/float): Joint angle (rotation about z-axis)
        alpha (symbol/float): Link twist (rotation about x-axis)
        r (symbol/float): Link length (translation along x-axis)
        d (symbol/float): Link offset (translation along z-axis)
    
    Returns:
        sympy.Matrix: The 4x4 DH transformation matrix
    """
    T = sp.Matrix([
        [sp.cos(theta), -sp.sin(theta) * sp.cos(alpha), sp.sin(theta) * sp.sin(alpha), r * sp.cos(theta)],
        [sp.sin(theta), sp.cos(theta) * sp.cos(alpha), -sp.cos(theta) * sp.sin(alpha), r * sp.sin(theta)],
        [0, sp.sin(alpha), sp.cos(alpha), d],
        [0, 0, 0, 1]
    ])
    
    return T

# Example evaluation with given values
t1, t2, t3, alpha, r, d = sp.symbols('theta1, theta2, theta3, alpha r d')
T1_2 = dh_transformation(t1, sp.rad(-90), d=0.163, r=0)
T2_3 = dh_transformation(t2, 0, d=0, r=0.425)
T3_rf = dh_transformation(t3, 0, d=0, r=0.392)

In [15]:
Ta_ef = np.array([[-1.0000000e+00,  1.2246468e-16,  0.0000000e+00,  0.0000000e+00],
       [-1.2246468e-16, -1.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  1.0000000e+00,  1.0000000e-01],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00]])



T1_rf = T1_2*T2_3*T3_rf*Ta_ef
T1_rf

Matrix([
[1.0*sin(theta2)*sin(theta3)*cos(theta1) + 1.2246468e-16*sin(theta2)*cos(theta1)*cos(theta3) + 1.2246468e-16*sin(theta3)*cos(theta1)*cos(theta2) - 1.0*cos(theta1)*cos(theta2)*cos(theta3), -1.2246468e-16*sin(theta2)*sin(theta3)*cos(theta1) + 1.0*sin(theta2)*cos(theta1)*cos(theta3) + 1.0*sin(theta3)*cos(theta1)*cos(theta2) + 1.2246468e-16*cos(theta1)*cos(theta2)*cos(theta3), -1.0*sin(theta1), -0.1*sin(theta1) - 0.392*sin(theta2)*sin(theta3)*cos(theta1) + 0.392*cos(theta1)*cos(theta2)*cos(theta3) + 0.425*cos(theta1)*cos(theta2)],
[1.0*sin(theta1)*sin(theta2)*sin(theta3) + 1.2246468e-16*sin(theta1)*sin(theta2)*cos(theta3) + 1.2246468e-16*sin(theta1)*sin(theta3)*cos(theta2) - 1.0*sin(theta1)*cos(theta2)*cos(theta3), -1.2246468e-16*sin(theta1)*sin(theta2)*sin(theta3) + 1.0*sin(theta1)*sin(theta2)*cos(theta3) + 1.0*sin(theta1)*sin(theta3)*cos(theta2) + 1.2246468e-16*sin(theta1)*cos(theta2)*cos(theta3),  1.0*cos(theta1), -0.392*sin(theta1)*sin(theta2)*sin(theta3) + 0.392*sin(theta1)*c

In [16]:
T1_rf[:3,3]

Matrix([
[-0.1*sin(theta1) - 0.392*sin(theta2)*sin(theta3)*cos(theta1) + 0.392*cos(theta1)*cos(theta2)*cos(theta3) + 0.425*cos(theta1)*cos(theta2)],
[-0.392*sin(theta1)*sin(theta2)*sin(theta3) + 0.392*sin(theta1)*cos(theta2)*cos(theta3) + 0.425*sin(theta1)*cos(theta2) + 0.1*cos(theta1)],
[                                              -0.392*sin(theta2)*cos(theta3) - 0.425*sin(theta2) - 0.392*sin(theta3)*cos(theta2) + 0.163]])

In [17]:
import numpy as np

def forward_kinematics_np(t1, t2, t3):
    """Compute the forward kinematics transformation."""
    
    x = -0.392 * np.sin(t2) * np.sin(t3) * np.cos(t1) + 0.392 * np.cos(t1) * np.cos(t2) * np.cos(t3) + 0.425 * np.cos(t1) * np.cos(t2)
    y = -0.392 * np.sin(t1) * np.sin(t2) * np.sin(t3) + 0.392 * np.sin(t1) * np.cos(t2) * np.cos(t3) + 0.425 * np.sin(t1) * np.cos(t2)
    z = -0.392 * np.sin(t2) * np.cos(t3) - 0.425 * np.sin(t2) - 0.392 * np.sin(t3) * np.cos(t2) + 0.163
    
    return np.array([x, y, z])

# Example usage:
t1, t2, t3 = np.radians([0, 0, 0])  # Convert degrees to radians
position = forward_kinematics_np(t1, t2, t3)
print(position)


[0.817 0.    0.163]
